In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from IPython import display

In [2]:
#Our standard sinusoid generator function
def f(x):
    return np.sin(2.0 * np.pi * x)

In [3]:
#Get m = 50 points
np.random.seed(9)
m = 50

#x_plot is evenly spaced points to plot the generator function
x_plot = np.linspace(0, 1, m)

#unevenly spaced points to generate the training data
X = np.random.uniform(0, 1, size=m)[:, np.newaxis] 
y = f(X) + np.random.normal(scale=0.3, size=m)[:, np.newaxis]

In [4]:
#The standard plot of the generator function and the training points

mpl.style.use('ggplot')
fig = plt.figure(figsize=(10,5))
ax = plt.subplot(111)
ax.plot(x_plot, f(x_plot), 'g')
ax.set_xlim(0,1)
ax.set_ylim(-2,2)
ax.set_xlabel('X')
ax.set_ylabel('y')
ax.set_title('Training points')
ax.scatter(X, y)

In [5]:
#Two useful plotting functions, plot_approximation will plot the model predictions, and plot_model_parameters will
#plot the values of the vector theta. 

def plot_approximation(xplot, X, y, clf, ax, llabel=None, show_test=0):
    '''Plot the approximation of clf on axis ax'''
    
    #plot the generator function
    ax.plot(x_plot, f(x_plot), 'g')
    
    #plot the training data
    ax.scatter(X, y, color='b')
    
    #plot the predictions from the model clf
    ax.plot(x_plot, clf.predict(x_plot[:, np.newaxis]), 'r', label=llabel)
    
    ax.set_ylabel('y')
    ax.set_xlabel('X')
    ax.set_ylim((-2, 2))
    ax.set_xlim((0, 1))
    ax.legend(loc='upper right')
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.2)
    
def plot_model_parameters(clf, ax, llabel=None, ylim_min=1e-1):
    "Plot the model parameters of clf on axis ax"
    
    #This prizes the theta parameters out of make pipeline!
    coef = clf.steps[-1][1].coef_.ravel()
    
    ax.set_yscale('Log')
    ax.set_ylim((ylim_min, 1e8))
    ax.set_ylabel('Absolute Value Model Parameter (Log))')
    ax.set_xlabel('Model Parameter')
    ax.set_xlim((1, 10))
    
    #plot the absolute values of the model parameters
    ax.plot(np.abs(coef), marker='o', label=llabel)
    
    #Also print out the model parameters
    print "Model Parameters:"
    mp_list = clf.steps[-1][1].coef_.ravel()
    for i in xrange(1, len(mp_list)):
        print "{:7.2f}".format(mp_list[i]),
    print "\n"

---
#The problem of over-fitting
---

##If there are too many features then the hypothesis function may fit the training set extremely well, but fail to 'generalize' well to new examples
##Size of the model - meaning the number and complexity of the features contribute to the model fit

---
#Model Fit:
---
## 1. Under-fit. The model is too simple and cannot fit the training data. Poor generalization. Called "High Bias"
## 2. Over-fit. The model is overly complex and fits the training data extremely well. Poor generalization. "High Variance"

---
#Let's illustrate Bias and Variance
---

In [7]:
fig = plt.figure(figsize=(20,10))
ax1 = plt.subplot(221)
ax2 = plt.subplot(222)
ax3 = plt.subplot(223)
ax4 = plt.subplot(224)
axes = [ax1, ax2, ax3, ax4]

clf_list = []

#Build and fit model of increasing degree polynomials, using simple Least Squares Regression
for axis, degree in enumerate([1, 3, 9, 15]):
    clf = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    clf_list.append(clf)
    clf.fit(X, y)
    
    #Plot the generator function, the training data, and the model predictions
    plot_approximation(x_plot, X, y, clf, axes[axis],\
                       llabel = "Degree = {:d}".format(degree), show_test=0)

---
#Generalization
---
##The training data is split into 2 subsets - training set, and the validation set
##Models of increasing complexity are fitted to the training set
##The Mean Squared Error is measured for the model predictions on the training set and the (unseen) validation set
##Note that as the complexity or size of the model increases the MSE on the training set continues to fall
##BUT the MSE on the validation set reaches a minimum and then rises again. The model is failing to generalize on unseen data as it over-fits

---
#Generalization, Bias, Variance as a function of the degree of the polynomial
---

In [8]:
np.random.seed(9)
m = 100
#x_plot = np.linspace(0, 1, m)
X = np.random.uniform(0, 1, size=m)[:, np.newaxis] 
y = f(X) + np.random.normal(scale=0.3, size=m)[:, np.newaxis]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

max_degree = 10
training_error = np.zeros(max_degree)
testing_error = np.zeros(max_degree)

for degree in xrange(max_degree):
    clf = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    clf.fit(X_train, y_train)
    training_error[degree] = mean_squared_error(y_train, clf.predict(X_train))
    testing_error[degree] = mean_squared_error(y_test, clf.predict(X_test))


fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)
ax.plot(np.arange(10), training_error, 'g', label='Training Error')
ax.plot(np.arange(10), testing_error, 'r', label='Validation Error')
ax.set_yscale('Log')
ax.set_xlabel('Degree')
ax.set_ylabel('Log(MSE)')
ax.legend(loc='lower left')

---
#Over-fitting
---

##Look what happens to the magnitudes of the model parameters
---

In [9]:
fig = plt.figure(figsize=(20,40))
ax1 = plt.subplot(421)
ax2 = plt.subplot(422)
ax3 = plt.subplot(423)
ax4 = plt.subplot(424)
ax5 = plt.subplot(425)
ax6 = plt.subplot(426)
ax7 = plt.subplot(427)
ax8 = plt.subplot(428)
axes = [[ax1, ax2], [ax3, ax4], [ax5, ax6], [ax7, ax8]]
alphas = [0.0, 1e-8, 1e-5, 1e-1]

for i, degree in enumerate([2, 4, 8, 10]):
    left_axes = axes[i][0]
    right_axes = axes[i][1]
    clf = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    clf.fit(X_train, y_train)
    plot_approximation(x_plot, X_train, y_train, clf, left_axes, llabel = "Degree = {:d}".format(degree))
    plot_model_parameters(clf, right_axes, llabel=None)

---
#Addressing the over-fitting problem
---
## * Plot the hypothesis
###     - usually of limited value because we usually have a lot of features

---

## * Reduce the number of features
###     - select which features to keep, and which to throw out
###     - but maybe all of the features are useful. Throwing away features may not be a good idea

---

## * Regularization
###     - Keep all/most of the features, but reduce the magnitudes of the parameters of the model ($\theta$s)
###     - Works well when we have a lot of features, all of which might contribute a little towards predicting $y$
---

---
#Regularization
---

##What would happen if we were to add to our sum of squares cost function the following terms?
## $$J(\theta)=\frac{1.0}{2m}\left[\sum_{i=1}^{m}(y_{i}-\hat{y}_{i})^2 + 1000 * \theta_{3} + 1000 * \theta_{4}\right]$$
##In addition to minimizing the sum of squares term, $\theta_{3}$ and $\theta_{4}$ would both be driven towards $0$

---

##More generally: $$J(\theta)=\frac{1.0}{2m}\left[\sum_{i=1}^{m}(y_{i}-\hat{y}_{i})^2 + \lambda\sum_{j}^{N}\theta_{j}\right]$$

---

##By adding an extra term that penalizes large model parameters we can produce models with small parameter values, which prevents over-fitting

---
#Intuition
---
##Small parameter values:
###Produce a 'less complex' hypothesis function - 'smoother'
###Produce a linear model that is less prone to over-fitting
###One approach is to shrink all the parameters - all the $\theta$s, using a single regularization parameter $\lambda$
###$$J(\theta)=\frac{1.0}{2m}\left[\sum_{i=1}^{m}(y_{i}-\hat{y}_{i})^2 + \lambda\sum_{j}^{N}\theta_{j}\right]$$
###If $\lambda$ is set too high you will under-fit (again)!! - straight line
---

---
#Types of regularization
---

##There are different types of regularization terms available, and these are built into different regression algorithms:

## 1. Ridge (Tikhonov regularization) = L2-norm = Euclidean norm of the sum of the parameters, $\theta$, of the model $= \lambda||\theta||^{2}$
###As the penalty is increased ALL parameters shrink, while still remaining non-zero

---

## 2. Lasso (Least Absolute Shrinkage and Selection Operator) = L1-norm $= \lambda||\theta||$
###As the penalty is increased MORE of the parameters will shrink to zero
###This can discard features

---

## 3. Elastic Net. A linear combination of Ridge and Lasso

---

#Sklearn - the regularization hyper-parameter is $\alpha$, alpha

#Ridge Regression

In [11]:
#same routine as above except we use Ridge rather than simple least squares regression
fig = plt.figure(figsize=(20,40))
ax1 = plt.subplot(421)
ax2 = plt.subplot(422)
ax3 = plt.subplot(423)
ax4 = plt.subplot(424)
ax5 = plt.subplot(425)
ax6 = plt.subplot(426)
ax7 = plt.subplot(427)
ax8 = plt.subplot(428)
axes = [[ax1, ax2], [ax3, ax4], [ax5, ax6], [ax7, ax8]]

#Let's see the effect of regularizing with the following parameters
alphas = [0.0, 1e-8, 1e-5, 1e-1]

#set the polynomial to an arbitrarily high degree
degree=9

for axis, aalpha in enumerate(alphas):
    left_axes = axes[axis][0]
    right_axes = axes[axis][1]
    
    #Use Ridge
    clf = make_pipeline(PolynomialFeatures(degree), Ridge(alpha = aalpha))
    clf.fit(X_train, y_train)
    plot_approximation(x_plot, X_train, y_train, clf, \
                       left_axes, llabel = "Alpha = {:0.8f}".format(aalpha))
    plot_model_parameters(clf, right_axes, llabel=None)
    #print clf.steps[-1][1].coef_

---
#Lasso Regression
---

In [14]:
#Same routine as above, except we will use Lasso regression
fig = plt.figure(figsize=(20,40))
ax1 = plt.subplot(421)
ax2 = plt.subplot(422)
ax3 = plt.subplot(423)
ax4 = plt.subplot(424)
ax5 = plt.subplot(425)
ax6 = plt.subplot(426)
ax7 = plt.subplot(427)
ax8 = plt.subplot(428)
axes = [[ax1, ax2], [ax3, ax4], [ax5, ax6], [ax7, ax8]]

#Here is the list of regularization hyper-parameters we will try
alphas = [1e-10, 1e-8, 1e-5, 1e-2]
#Set an arbitrarily high degree polynomial
degree=9

for axis, aalpha in enumerate(alphas):
    left_axes = axes[axis][0]
    right_axes = axes[axis][1]
    
    #Use Lasso
    clf = make_pipeline(PolynomialFeatures(degree), Lasso(alpha = aalpha))
    clf.fit(X_train, y_train)
    plot_approximation(x_plot, X_train, y_train, clf, \
                       left_axes, llabel = "Alpha = {:0.8f}".format(aalpha))
    plot_model_parameters(clf, right_axes, llabel=None, ylim_min=0.1)

---
#Regularized Linear Regression (gradient descent and linear equations)
---
## * In gradient descent the addition of the regularization term shrinks the parameters on each iteration
## * In the normal equations a similar thing happens in the math analytically
 
--- 
 ##Sklearn Ridge/Lasso/ElasticNet Regression chooses internally how it will fit the linear model - iterative or analytic. Mostly they appear to use iterative gradient descent-like algorithms to solve for the linear model parameters
 ---
 ##Sklearn Lasso is identical to simple least squares LinearRegression() if $\alpha = 0$.
 ---

---
#Generalization, Bias, Variance as a function of the Regularization Parameter
---

In [17]:
#Similar to the Generalization, Bias, Variance as a function of degree of polynomial plot above
np.random.seed(9)
m = 100
X = np.random.uniform(0, 1, size=m)[:, np.newaxis] 
y = f(X) + np.random.normal(scale=0.3, size=m)[:, np.newaxis]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

#This time set up a very large complex polynomial
max_degree = 25

#We will investigate the following regularization hyper-parameters
alphas = [0.0, 1e-8, 1e-6, 1e-4, 1e-2, 1, 100]

training_error = []
testing_error = []

for a in alphas:
    #Use Ridge as an example
    clf = make_pipeline(PolynomialFeatures(max_degree), Ridge(alpha=a))
    
    #Fit the model
    clf.fit(X_train, y_train)
    
    #Accumulate the errors
    training_error.append(mean_squared_error(y_train, clf.predict(X_train)))
    testing_error.append(mean_squared_error(y_test, clf.predict(X_test)))

for i in xrange(len(training_error)):
    print "{:18.5f}".format(training_error[i]),
    
print "\n"

for i in xrange(len(testing_error)):
    print "{:18.5f}".format(testing_error[i]),

print "\n"

#Plot the training and test errors against the value of the regularization hyper-pararmeter    
fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)
ax.plot(np.array(alphas), np.array(training_error), color = 'green', marker = 'o', label='Training Error')
ax.plot(np.array(alphas), np.array(testing_error), color = 'red', marker = 'o', label='Validation')
ax.set_yscale('Log')
ax.set_ylim(0.01, 100000)
ax.set_xscale('Log')
ax.set_xlabel('Lambda (Alpha)')
ax.set_ylabel('Log(MSE)')
ax.legend(loc='upper right')